In [5]:
import numpy  as np
import pandas as pd
import lightkurve as lk

from scipy.stats import kurtosis

c:\Users\jm_ra\OneDrive - Universidad de Concepción\Projectos\NASA_SPACE\venv\Lib\site-packages\lightkurve\prf\__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


In [6]:
import os

In [7]:
def _select_windows(phase, flux, width):
    """Devuelve máscaras para: in-transit (|phi|<=w) y out-of-transit (|phi|>=3w)."""
    in_tr = np.abs(phase) <= width
    oot   = np.abs(phase) >= 3*width
    return in_tr, oot

def _baseline_depth(phase, flux, width):
    """Estima baseline (OOT) y profundidad del tránsito."""
    in_tr, oot = _select_windows(phase, flux, width)
    if np.sum(oot) < 10:  # fallback si no hay suficiente OOT
        baseline = np.median(flux)
    else:
        baseline = np.median(flux[oot])
    depth = baseline - np.min(flux[in_tr]) if np.any(in_tr) else 0.0
    if depth <= 0:
        depth = np.ptp(flux) * 0.05 + 1e-9  # evita divisiones por cero
    return baseline, depth

In [8]:
def symmetry_index(phase, flux, width=0.05, nbins=100):
    phase = np.asarray(phase)
    flux  = np.asarray(flux)

    # Ventana del tránsito
    sel = np.abs(phase) <= width
    if np.sum(sel) < 20:
        return np.nan

    # Bin en |phi|
    abs_phi = np.abs(phase[sel])
    f_sel   = flux[sel]

    bins = np.linspace(0, width, nbins+1)
    idx  = np.digitize(abs_phi, bins) - 1
    
    # Para cada bin, calculamos mediana en lado negativo y positivo
    diffs = []
    for b in range(nbins):
        mask_bin = idx == b
        if not np.any(mask_bin):
            continue
        phi_bin  = phase[sel][mask_bin]
        f_bin    = f_sel[mask_bin]
        neg = (phi_bin < 0); pos = (phi_bin > 0)
        if np.any(neg) and np.any(pos):
            f_neg = np.median(f_bin[neg])
            f_pos = np.median(f_bin[pos])
            diffs.append(np.abs(f_pos - f_neg))

    if len(diffs) == 0:
        return np.nan

    # Normalizamos por la profundidad para hacerlo adimensional
    _, depth = _baseline_depth(phase, flux, width)
    return np.median(diffs) / depth

In [9]:
def v_u_kurtosis(phase, flux, width=0.05):

    phase = np.asarray(phase)
    flux  = np.asarray(flux)

    in_tr, oot = _select_windows(phase, flux, width)

    if np.sum(in_tr) < 20:
        return np.nan

    baseline, depth = _baseline_depth(phase, flux, width)
    x = (baseline - flux[in_tr]) / depth
    # scipy.stats.kurtosis por defecto devuelve “exceso de kurtosis” (Fisher=True).
    # Ponemos Fisher=False para que la gaussiana tenga 3 (más intuitivo).
    return kurtosis(x, fisher=False, bias=False)

In [10]:
def ingress_egress_slopes(phase, flux, width=0.05):
    phase = np.asarray(phase)
    flux  = np.asarray(flux)

    # Ingreso: (-w, 0)
    ing = (phase >= -width) & (phase < 0)
    egr = (phase > 0) & (phase <= +width)

    m_ing, m_egr = np.nan, np.nan
    if np.sum(ing) >= 5:
        m_ing = np.polyfit(phase[ing], flux[ing], 1)[0]
    if np.sum(egr) >= 5:
        m_egr = np.polyfit(phase[egr], flux[egr], 1)[0]
    return m_ing, m_egr

In [ ]:
def metrics_from_folded_lc(fold_lc, width=0.1):
    phase = np.asarray(getattr(fold_lc, "phase").value if hasattr(fold_lc, "phase") else fold_lc.phase)
    flux  = np.asarray(getattr(fold_lc, "flux").value  if hasattr(fold_lc, "flux")  else fold_lc.flux)

    asym = symmetry_index(phase, flux, width=width)
    kurt = v_u_kurtosis(phase, flux, width=width)
    m_ing, m_egr = ingress_egress_slopes(phase, flux, width=width)

    return {
        "asymmetry_index": asym,
        "v_u_kurtosis":    kurt,
        "ingress_slope":   m_ing,
        "egress_slope":    m_egr,
    }

['KIC-10015516.fits', 'KIC-10118816.fits', 'KIC-10122255.fits']